In [1]:
import string
import requests
import json
from pprint import pprint


In [2]:
abecedario = list(string.ascii_lowercase)
print(abecedario)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [3]:
TORRE_API_URL = "http://arda.torre.co/entities/_searchStream"
LIMIT_SEARCH = 10000
all_responses = []
for letra in abecedario:

  external_request_body = {
      "query": letra,
      "meta": False,
      "limit": LIMIT_SEARCH,
      "torreGgId": "2076714",
      "excludeContacts": True,
      "excludedPeople": []
  }
  response = requests.post(TORRE_API_URL, json=external_request_body)

  if response.status_code == 200:
    for raw_line in response.iter_lines(decode_unicode=True):
      all_responses.append(json.loads(raw_line))


In [4]:
len(all_responses)

124630

In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.DataFrame(all_responses)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124630 entries, 0 to 124629
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ardaId                 124630 non-null  int64  
 1   ggId                   116828 non-null  object 
 2   name                   124630 non-null  object 
 3   comparableName         124630 non-null  object 
 4   username               116828 non-null  object 
 5   professionalHeadline   100699 non-null  object 
 6   imageUrl               65578 non-null   object 
 7   completion             116828 non-null  float64
 8   grammar                116828 non-null  float64
 9   weight                 116828 non-null  float64
 10  verified               116828 non-null  object 
 11  connections            124630 non-null  object 
 12  totalStrength          124630 non-null  float64
 13  pageRank               124630 non-null  float64
 14  organizationId         7802 non-null

In [8]:
df.head(1)

,ardaId,ggId,name,comparableName,username,professionalHeadline,imageUrl,completion,grammar,weight,...,totalStrength,pageRank,organizationId,organizationNumericId,publicId,status,creators,relationDegree,isSearchable,contact
0,41469759,None,AI Solutions,ai solutions,None,None,https://res.cloudinary.com/torre-technologies-...,NaN,NaN,NaN,...,1.05,3.494235,xZdOEXVq,1983201,AISolutions1,pending,[1530379],1,True,False


In [11]:
df["username"].drop_duplicates().dropna().to_json("all_usernames.json", orient="records")

In [17]:
user_names = df["username"].drop_duplicates().dropna().values

In [18]:
import requests
import pandas as pd
import json
import os
from datetime import datetime
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# --- CONFIGURACIÓN ---
# Define cuántas peticiones simultáneas quieres hacer.
# Empieza con un número bajo (ej. 10) y auméntalo con cuidado para no saturar la API.
MAX_WORKERS = 10 
# Archivo donde se guarda el resultado y el progreso
PROGRESS_FILE = "bios_recolectados.json"


# --- 1. FUNCIÓN PARA PROCESAR UN ÚNICO USUARIO ---
# Aislamos la lógica de una petición en una función. Esto es clave para la paralelización.
def fetch_bio(user):
    """Realiza la petición para un solo usuario y devuelve el resultado o None si hay error."""
    url = f"https://torre.ai/api/genome/bios/{user}"
    headers = {
        'Cookie': 'trackingId=rB8YnWhMteJzXQsgDGHRAg=='
    }
    try:
        # Añadimos un timeout para evitar que una petición se quede colgada indefinidamente
        response = requests.get(url, headers=headers, timeout=15)

        if response.status_code == 200:
            return response.json()
        else:
            # Devolvemos el error junto con el usuario para poder registrarlo si es necesario
            return {"error": True, "status_code": response.status_code, "user": user}
            
    except requests.RequestException as e:
        # Errores de red, timeout, etc.
        return {"error": True, "exception": str(e), "user": user}

# --- 2. CARGA DE DATOS Y PROGRESO ---
# Esta parte es idéntica a tu código original, lo cual es perfecto.
try:
    # Asumiendo que 'df' ya está cargado en tu entorno. Si no, cárgalo aquí.
    # df = pd.read_csv('tu_archivo.csv') 
    user_names = df["username"].drop_duplicates().dropna().values
except NameError:
    print("Error: La variable 'df' no está definida. Por favor, carga tu DataFrame primero.")
    # Crea una lista de ejemplo si no hay df, para que el script pueda correr
    user_names = [f"user{i}" for i in range(100)] 

# Cargar bios ya recolectadas (si existe)
if os.path.exists(PROGRESS_FILE):
    print(f"Cargando progreso desde {PROGRESS_FILE}...")
    with open(PROGRESS_FILE, "r", encoding='utf-8') as f:
        # Usamos un try-except por si el JSON está vacío o corrupto
        try:
            all_users_bio = json.load(f)
            # Extraemos los usernames ya procesados correctamente
            processed_users = {
                user["person"]["username"] 
                for user in all_users_bio 
                if isinstance(user, dict) and "person" in user and "username" in user["person"]
            }
        except json.JSONDecodeError:
            all_users_bio = []
            processed_users = set()
    print(f"Se encontraron {len(processed_users)} usuarios ya procesados.")
else:
    all_users_bio = []
    processed_users = set()

# Filtrar usuarios que faltan por procesar
users_to_process = [user for user in user_names if user not in processed_users]
print(f"Total de usuarios a procesar: {len(users_to_process)}")

if not users_to_process:
    print("No hay nuevos usuarios que procesar. ¡Trabajo terminado!")
else:
    # --- 3. EJECUCIÓN EN PARALELO ---
    try:
        # Usamos un 'with' para asegurarnos de que los threads se cierren correctamente
        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            
            # Creamos un "futuro" para cada petición. Esto las pone en cola para ejecución.
            future_to_user = {executor.submit(fetch_bio, user): user for user in users_to_process}
            
            # Usamos tqdm para crear una barra de progreso que se actualiza a medida que las tareas terminan
            pbar = tqdm(as_completed(future_to_user), total=len(users_to_process), desc="Procesando bios en paralelo")
            
            for future in pbar:
                result = future.result()
                if result:
                    # Si el resultado no es un error, lo añadimos a la lista
                    if not result.get("error"):
                        all_users_bio.append(result)
                    else:
                        # Opcional: imprimir los errores
                        user_failed = result.get("user")
                        pbar.write(f"Error al procesar {user_failed}: {result}")

    except KeyboardInterrupt:
        print("\nProceso interrumpido por el usuario. Guardando progreso...")
    except Exception as e:
        print(f"Se produjo un error general: {e}")
    finally:
        # --- 4. GUARDADO FINAL ---
        # Este bloque se ejecuta siempre: si termina bien, si hay un error o si lo interrumpes.
        print(f"\nGuardando {len(all_users_bio)} bios recolectadas en total.")
        with open(PROGRESS_FILE, "w", encoding='utf-8') as f:
            json.dump(all_users_bio, f, indent=2, ensure_ascii=False)
        print(f"Progreso guardado en {PROGRESS_FILE}")

Total de usuarios a procesar: 68210


Procesando bios en paralelo:   0%|          | 152/68210 [00:23<4:47:58,  3.94it/s]

Error al procesar nicholasandrewmontes: {'error': True, 'exception': "HTTPSConnectionPool(host='torre.ai', port=443): Read timed out. (read timeout=15)", 'user': 'nicholasandrewmontes'}


Procesando bios en paralelo:   1%|          | 727/68210 [01:32<1:42:42, 10.95it/s]

Error al procesar robertocarlosarangosuarez: {'error': True, 'exception': "HTTPSConnectionPool(host='torre.ai', port=443): Read timed out. (read timeout=15)", 'user': 'robertocarlosarangosuarez'}


...

Procesando bios en paralelo:   2%|▏         | 1223/68210 [03:08<6:02:57,  3.08it/s]

Error al procesar carlosfranciscoalvarezgallardo: {'error': True, 'exception': "HTTPSConnectionPool(host='torre.ai', port=443): Read timed out. (read timeout=15)", 'user': 'carlosfranciscoalvarezgallardo'}


Procesando bios en paralelo:   2%|▏         | 1232/68210 [03:11<7:07:54,  2.61it/s]

Error al procesar deibisalexandersanchezfreitez: {'error': True, 'exception': "HTTPSConnectionPool(host='torre.ai', port=443): Read timed out. (read timeout=15)", 'user': 'deibisalexandersanchezfreitez'}


Procesando bios en paralelo:   2%|▏         | 1381/68210 [03:34<2:53:13,  6.43it/s]


<Response [404]>

In [74]:
all_users_bio

[{'person': {'professionalHeadline': 'CEO @ Torre.ai. Mod of the /nearshore and /remoto subtorres.',
   'completion': 0.875,
   'showPhone': False,
   'created': '2017-06-25T02:49:44Z',
   'verified': True,
   'flags': {'accessCohort': False,
    'benefits': True,
    'canary': True,
    'enlauSource': False,
    'fake': False,
    'featureDiscovery': False,
    'firstSignalSent': True,
    'signalsOnboardingCompleted': True,
    'importingLinkedin': False,
    'onBoarded': True,
    'remoter': True,
    'signalsFeatureDiscovery': False,
    'importingLinkedinRecommendations': False,
    'contactsImported': True,
    'appContactsImported': True,
    'genomeCompletionAcknowledged': True,
    'cvImported': False,
    'communityCreatedPrivate': False,
    'communityCreatedClaimed': False,
    'connectBenefitsViewed': True,
    'recommendationLeadEmailSent': True,
    'recommendationsAskedGenomeCompletion': True,
    'behavioralTraitsAcknowledged': True,
    'testTaken': True,
    'preview